In [26]:
from keras.callbacks import LambdaCallback
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
import random
import sys
import io
import os

In [33]:
path = './data/names.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

In [36]:
text = text.replace("'", "")
chars = sorted(list(set(text)))
print('total chars:', len(chars))
print(chars)

total chars: 71
['\n', ' ', '-', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', 'ʻ', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'і', 'ғ', 'қ', 'ң', 'ү', 'ұ', 'һ', 'ә', 'ө']


In [37]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)

{'\n': 0, ' ': 1, '-': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'x': 25, 'y': 26, 'z': 27, 'ʻ': 28, 'а': 29, 'б': 30, 'в': 31, 'г': 32, 'д': 33, 'е': 34, 'ж': 35, 'з': 36, 'и': 37, 'й': 38, 'к': 39, 'л': 40, 'м': 41, 'н': 42, 'о': 43, 'п': 44, 'р': 45, 'с': 46, 'т': 47, 'у': 48, 'ф': 49, 'х': 50, 'ц': 51, 'ч': 52, 'ш': 53, 'щ': 54, 'ъ': 55, 'ы': 56, 'ь': 57, 'э': 58, 'ю': 59, 'я': 60, 'ё': 61, 'і': 62, 'ғ': 63, 'қ': 64, 'ң': 65, 'ү': 66, 'ұ': 67, 'һ': 68, 'ә': 69, 'ө': 70}


In [5]:
ix_to_char={i:ch for i,ch in enumerate(chars)}
char_to_ix={ch:i for i,ch in enumerate(chars)}
print(char_to_ix)

{' ': 1, '"': 2, '-': 3, ':': 4, ';': 5, 'a': 6, 'b': 7, 'c': 8, 'd': 9, 'e': 10, 'f': 11, 'g': 12, 'h': 13, 'i': 14, 'j': 15, 'k': 16, 'l': 17, 'm': 18, 'n': 19, 'o': 20, 'p': 21, 'q': 22, 'r': 23, 's': 24, 't': 25, 'u': 26, 'v': 27, 'x': 28, 'y': 29, 'z': 30, 'ʻ': 31, 'а': 32, 'б': 33, 'в': 34, 'г': 35, 'д': 36, 'е': 37, 'ж': 38, 'з': 39, 'и': 40, 'й': 41, 'к': 42, 'л': 43, 'м': 44, 'н': 45, 'о': 46, 'п': 47, 'р': 48, 'с': 49, 'т': 50, 'у': 51, 'ф': 52, 'х': 53, 'ц': 54, 'ч': 55, 'ш': 56, 'щ': 57, 'ъ': 58, 'ы': 59, 'ь': 60, 'э': 61, 'ю': 62, 'я': 63, 'ё': 64, 'і': 65, 'ғ': 66, 'қ': 67, 'ң': 68, 'ү': 69, 'ұ': 70, 'һ': 71, 'ә': 72, 'ө': 73}


In [38]:
lines = text.split('\n')
lines = [line for line in lines if len(line)!=0]
print("number of lines:", len(lines))

number of lines: 56059


In [39]:
maxlen = len(max(lines, key=len))
minlen = len(min(lines, key=len))

print("line with longest length: "+ str(maxlen))
print("line with shorter length: "+ str(minlen))

line with longest length: 41
line with shorter length: 7


In [40]:
steps = 1
sequences = []
next_chars = []

for line in lines:
    # pre-padding with zeros
    s = (maxlen - len(line))*'0' + line
    sequences.append(s)
    next_chars.append('\n')
    for it,j in enumerate(line):
        if (it >= len(line)-1):
            continue
        s = (maxlen - len(line[:-1-it]))*'0' + line[:-1-it]
        sequences.append(s)
        next_chars.append(line[-1-it])

In [41]:
print('total sequences:', len(sequences))
print(sequences[66], next_chars[66])
print(sequences[67], next_chars[67])
print(sequences[68], next_chars[68])

total sequences: 1421295
00000000000000000000000000ерболатова акбо т
000000000000000000000000000ерболатова акб о
0000000000000000000000000000ерболатова ак б


In [42]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        if char != '0':
            x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

/var/folders/8h/xg2d0hkn4xng7c08vc_2khgr0000gn/T/ipykernel_12366/4182269958.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
/var/folders/8h/xg2d0hkn4xng7c08vc_2khgr0000gn/T/ipykernel_12366/4182269958.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sequences)

In [45]:
prefix = ""
max_names = 10

def sample(preds):
    """ function that sample an index from a probability array """
    preds = np.asarray(preds).astype('float64')
    preds = preds / np.sum(preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.random.choice(range(len(chars)), p = probas.ravel())

def print_name_generated(name):
    print(name, flush=True)
def print_list_generated(lst):
    print(lst, flush=True)


def generate_new_names(*args):
    print("----------Generatinig names----------")

    # Add pre-padding of zeros in the input.
    sequence = ('{0:0>' + str(maxlen) + '}').format(prefix).lower()

    # tmp variables
    tmp_generated = prefix
    list_outputs = list()

    while len(list_outputs) < max_names:

        # Vectorize the input of the model.
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sequence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1

        # Predict the probabilities of the next char.
        preds = model.predict(x_pred, verbose=0)[0]

        # Chose one based on the distribution obtained in the output of the model.
        next_index = sample(preds)
        # Get the corresponding char.
        next_char = indices_char[next_index]

        # If the char is a new line character or the name start to be bigger than the longest word,
        # try to add it to the list and reset temp variables.
        if next_char == '\n' or len(tmp_generated) > maxlen:

            # If the name generated is not in the list, append it and print it.
            if tmp_generated not in list_outputs:
                list_outputs.append(tmp_generated)
                print_name_generated(tmp_generated)
            # Reset tmp variables
            sequence = ('{0:0>' + str(maxlen) + '}').format(prefix).lower()
            tmp_generated = prefix
        else:

            # Append the char to the sequence that we're generating.
            tmp_generated += next_char
            # Add pre-padding of zeros to the sequence generated and continue.
            sequence = ('{0:0>' + str(maxlen) + '}').format(tmp_generated).lower()

    # Show the intersection of the words generated and your dataset. .
    print("Set of words already in the dataset:")
    print_list_generated(set(lines).intersection(list_outputs))

    # Show the rate of how many repeated words you've created.
    total_repited = len(set(lines).intersection(list_outputs))
    total = len(list_outputs)
    print("Rate of total invented words: " + "{:.2f}".format((total-total_repited)/total))
    print("-----------------End-----------------")

# Function invoked at the end of each epoch. Prints generated names.
callback = LambdaCallback(on_epoch_end=generate_new_names)

In [46]:
# build and train the model
model = Sequential()
model.add(LSTM(64, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))
history = model.fit(x, y, batch_size=128, epochs=2, verbose=2, callbacks=[callback])


Epoch 1/2


/Users/cyber/anaconda3/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:140: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


----------Generatinig names----------
дорманов мадин әбілханұлы
тактемев жансалин болатжанович
загопилжай айгерим ұлыққызы
шодынулла нұртас бауыржанұлы
оразгаир бекадел ныркубайулы
эфильв марат улугович
вазбағолова жарсын сәженярқызы
киратовн андра александрович
әдылжанова темірлан маматұлы
биденова динара болатовна
Set of words already in the dataset:
set()
Rate of total invented words: 1.00
-----------------End-----------------
11104/11104 - 458s - loss: 1.3033 - 458s/epoch - 41ms/step
Epoch 2/2
----------Generatinig names----------
жұмабергенбекова ернұр қайратқызы
шуомохар дарина данияровна
черков витальдра евгеньевич
эруекова айша дариянқызы
 нұрсұлтан маратұлы
жайдар нұрай абдуғынұрланқызы
фарсов набыс назирыс сейсентович
гайратов назерке матжановна
мазадин ильяс алийглы
эроргеидина слеөмана думанқызы
Set of words already in the dataset:
set()
Rate of total invented words: 1.00
-----------------End-----------------
11104/11104 - 458s - loss: 1.1628 - 458s/epoch - 41ms/step
